In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import os
import pandas as pd
import torch.nn as nn
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split, Dataset

from tqdm import tqdm

from utils.london.link_loads import get_graph_attributes, df_to_graph, build_quarter_hour_data, add_missing_nodes
# import networkx as nx

In [3]:
folder_path = "data/london/"
# Get graph attributes, create dfs from csv, process Link column and ordered dfs by time
num_nodes, edge_index, node_mapping, dfs = get_graph_attributes(folder_path)

In [4]:
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.extend(df_qhrs)
    
graphs = [df_to_graph(df, edge_index) for df in graph_data]  # Un graphe par quart d'heure

In [5]:
class GraphSequenceDataset(Dataset):
    def __init__(self, graphs, window_size=4):
        self.graphs = graphs
        self.window_size = window_size
        if len(graphs) <= window_size:
            raise ValueError("Error : window_size should be higher than the number of graphs")
        
    def __len__(self):
        return len(self.graphs) - self.window_size

    def __getitem__(self, idx):
        # input_graphs prend en compte les window_size graphe précédent l'instant idx + le graphe à prèdire à l'instant idx
        input_graphs = self.graphs[idx : idx + self.window_size + 1]
        target = self.graphs[idx + self.window_size].y  # On prédit y du dernier graph
        return input_graphs, target 

In [6]:
# Définition des loaders
window_size = 4  # Nombre de pas de temps en entrée
train_size = int(0.8 * len(graphs))

train_dataset = GraphSequenceDataset(graphs[:train_size], window_size)  
test_dataset = GraphSequenceDataset(graphs[train_size:], window_size)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [14]:
class TGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, gru_hidden_dim):
        super(TGCN, self).__init__()
        self.gcn = GCNConv(node_features, hidden_dim)  # GCN
        self.gru = nn.GRU(hidden_dim, gru_hidden_dim, batch_first=True)  # GRU
        self.fc = nn.Linear(gru_hidden_dim, 1)  # Prédiction finale

    def forward(self, graph_seq):
        # window_size = len(graph_seq)  # Nombre de pas de temps
        # batch_size = graph_seq[0].x.shape[0]  # Nombre de nœuds

        spatial_features = []
        for graph in graph_seq:
            x = self.gcn(graph.x, graph.edge_index)  # GCN
            x = F.relu(x)
            spatial_features.append(x)

        spatial_features = torch.stack(spatial_features, dim=1)  # (batch, time, hidden_dim)

        _ , final_state = self.gru(spatial_features)  # prédiction sur le dernier (1, gru_hidden_dim)
        final_state = final_state.squeeze() # (gru_hidden_dim)
        final_out = self.fc(final_state) # Prédiction sur le dernier état
        final_out = F.relu(final_out)
        return final_out

In [15]:
class CustomMAELoss(nn.Module):
    def __init__(self):
        super(CustomMAELoss, self).__init__()

    def forward(self, pred, target):
        return torch.mean(torch.abs(pred - target))

# Instanciation de la loss
MAE = CustomMAELoss()

In [ ]:
model = TGCN(node_features=10, hidden_dim=32, gru_hidden_dim=64)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = MAE

# Boucle d'entraînement
for epoch in tqdm(range(50)):
    model.train()
    total_loss = 0
    for graph_seq, target in train_loader:
        optimizer.zero_grad()
        output = model(graph_seq)

        target = target.squeeze()
        output = output.reshape(target.shape)


        print(output.shape)
        print(target.shape)
    
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")


  2%|▏         | 1/50 [01:30<1:13:54, 90.51s/it]

Epoch 1, Loss: 243.2198999875875


  4%|▍         | 2/50 [03:00<1:12:21, 90.45s/it]

Epoch 2, Loss: 228.43867672306067


  6%|▌         | 3/50 [04:25<1:08:55, 87.98s/it]

...

Epoch 40, Loss: 206.36833346556037


 82%|████████▏ | 41/50 [53:04<07:35, 50.66s/it]

Epoch 41, Loss: 206.36231648722926


 82%|████████▏ | 41/50 [53:22<11:42, 78.10s/it]


KeyboardInterrupt: 

In [19]:
model.eval()
test_loss = 0
with torch.no_grad():
    for graph_seq, target in test_loader:

        output = model(graph_seq)
        loss = criterion(output, target)
        test_loss += loss.item()

print(f"Test MAE: {test_loss / len(test_loader)}")


Test MAE: 204.33262727453592


In [22]:
39e6/(96*1206)

336.8573797678275

The average flow per inter_station per 15min is 337. So 204 MAE error is not a valid result.